In [1]:
# 현재 Ubuntu버전에서 다운로드받아 사용할 수 있는 MySQL 서버 버전을 검색
# sudo apt-cache search mysql-server

# 찾은 정보 중에 MySQL Server 5.7을 다운받는다.
# sudo apt-get install mysql-server-5.7

# 설치 마지막쯤 나오는 경로를 기억해두도록 한다. 나중에 쓸 일이 있는데 찾으려면 골아프다.
# update-alternatives:
# using /etc/mysql/mysql.cnf to provide 
# /etc/mysql/my.cnf (my.cnf) in auto mode
# Renaming removed key_buffer
# and myisam-recover options (if present)
# Created symlink
# /etc/systemd/system/multi-user.target.wants/mysql.service → /lib/systemd/system/mysql.service.

# 현재 MySQL 서버가 잘 동작 중인지 확인하는 방법
# service mysql status

# 현재 구동되고 있는 MySQL 서버 프로세스를 확인하는 방법
# ps -ef | grep mysql

# MySQL을 잡은 grep의 정보도 같이 보여주므로 걸러내기
# ps -f | grep mysql | grep -v grep

# netstat명령어를 사용하기 위해 필요한 SW툴을 설치
# sudo apt-get install net-tools

# 현재 구종중인 네트워크 서비스들을 확인해 볼 수 있다.
# sudo netstat -ntlp | grep mysql

# 12.0.0.1:3306
# 12.0.0.1이라는 주소는 localhost를 의미
# localhost란 현재 내 컴퓨터를 뜻한다.
# 12.0.0.1:3306이란 현재 내 컴퓨터에서 서비스되는 번호가 3306이라는 의미

# 만약 외부에서 해당내용을 볼 수 있게 만드려면
# ifconfig 명령어를 입력해서 현재 내 IP주소를 확인한다.

# MySQL에 루트계정으로 접속하는 방법
# sudo mysql -u root -p
# 학원비밀번호 입력
# 리눅스 세계에서는 root가 최강자
# 윈도우의 관리자 권한보다 상위라고 볼 수 있다. 루트권한을 잘못 사용하면 컴퓨터 아작남.

# Pythone에서 사용할 전용 DB 스키마를 만든다.
# create database pydb; 

# system clear DB에 Query를 작성하다 화면이 지저분해지면 지울 때 사용

# 일반적인 권한을 가진 사용자 계정을 만드는 방법
# create user 'bitai'@localhost identified by '456123';

# DB를 사용하기 위한 권한을 부여해줘야 한다.
# grant all privileges on *.* to 'bitai'@localhost identified by '456123';
# flush privileges;

# 앞서서 show detabases를 통해서 스키마들을 확인했다.
# use pydb; 어떤 스키마를 사용할지 결정한다.
# show tables; 현재 스키마에 어떤 DB테이블들이 구성되어있는지 보여준다.

# quit MySQL종료

# mysql -u bitai -p 생성한 bitai계정으로 MySQL에 접속

In [2]:
# pip install pymysql
import pymysql

# mysql -u bitai -p 비밀번호
# 위의 명령을 프로그래밍 버전으로 변형한 것임
db = pymysql.connect(
    # 접속할 컴퓨터의 주소
    host = '127.0.0.1',
    # 현재 구동중인 MySQL의 포트(서비스) 번호
    port = 3306,
    # 계정 이름
    user = 'bitai',
    # 비밀 번호
    passwd = '456123',
    # 사용하려는 스키마
    db = 'pydb'
)

# 후에 select 등을 했을때 결과를 하나 하나 커서를 내려가며 순회할 수 있는 녀석
cursor = db.cursor()

# 실제 MySQL에서 사용하는 DB Query 문
# create table 테이블명(원하는 필드와 원하는 타입을 작성)
# auto_increment는 자동 증가
# unsigned란 부호가 없다는 의미(즉 양수만 취급)
# varchar는 문자를 저장할 수 있는 배열
# primary key는 대표 키를 설정하는 키워드
sql = """
    create table pydbtest(
        id int unsigned not null auto_increment,
        name varchar(20) not null,
        price int not null,
        primary key(id)
    )
"""

# 위의 SQL Query를 실행하는 코드
# 작업을 메모리 상에서 해놓고 메모리에 올려놓고 있음
cursor.execute(sql)

# DB에 실제 내용을 갱신하는 코드
# 실제 이부분은 I/O로 작업을 내리는 과정
db.commit()

# 왜 실행하자마자 적용되지 않고 commit()이란 것이 별도로 있나요 ?
# 파이썬에서 join()을 배웠었는데 같은 이유다.
# 결론적으로 컴퓨터의 성능은 I/O와 직결된다.

# 작업을 완료했으므로 DB 연결을 끊는다.
db.close()

print("Table Creation Success")

Table Creation Success


In [3]:
# 위의 코드로 pydb스키마에 pydbtest라는 테이블을 생성함
# 확인은 MySQL에 접속해서 pydb스키마 사용하여 테이블 보기 후 내부 내용보기
# mysql -u bitai -p
# use pydb;
# show tables;
# describe pydbtest;

In [4]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor()

# insert into 테이블이름(필드 이름들 채움)
# 필드를 이름을 채울때 id처럼 auto_increment 되는 것은 제외한다.
# values (%s, %s)라고 되어 있는데
# 이 부분은 어떤 특정값을 가져와서 문자열 형태로 처리할 것이라느 의미
# DB 처리는 문자열로 처리하므로 %s를 보편적으로 사용한다.
sql = """
    insert into pydbtest(name, price) values (%s, %s)
"""

# 위의 %s가 두 개 있으므로
# sql로 전달되는 인자 또한 두개가 들어온다.
# execute할 때 sql의 %s에 인자로 넘어온 튜플이 들어가게 된다.
cursor.execute(sql, ('Xeon Server', 10000000))
cursor.execute(sql, ('FPGA Server', 34000000))
cursor.execute(sql, ('GPU Server', 20000000))
cursor.execute(sql, ('RTX Titan', 4000000))

db.commit()

db.close()

print("Insertion Success")

Insertion Success


In [5]:
# 입력값을 확인하는 방법
# mysql -u bitai -p로 접속한다.

# use pydb;
# select * from pydbtest;
# 위의 명령을 입력하면 pydbtest에 삽입된 값들을 볼 수 있다.

In [6]:

import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor()

cursor = db.cursor()

# pydbtest에 있는 모든것(*)을 선택해서 가져와라 ~
sql = """
    select * from pydbtest;
"""

cursor.execute(sql)
# cursor를 통해서 가져온 모든 데이터를 rows에 집어넣는다.
rows = cursor.fetchall()
print(rows)

db.close()

print("Selection Success")


((1, 'Xeon Server', 10000000), (2, 'FPGA Server', 34000000), (3, 'GPU Server', 20000000), (4, 'RTX Titan', 4000000))
Selection Success


In [7]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor()

sql = """
    select * from pydbtest;
"""

cursor.execute(sql)
rows = cursor.fetchall()
for row in rows:
    print(row)

db.close()

print("Selection Success")

(1, 'Xeon Server', 10000000)
(2, 'FPGA Server', 34000000)
(3, 'GPU Server', 20000000)
(4, 'RTX Titan', 4000000)
Selection Success


In [8]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

# 커서 정보를 Dictionary 형태로 사용하려면
# 아래의 코드를 반드시 구동해야 한다.
cursor = db.cursor(pymysql.cursors.DictCursor)

sql = """
    select * from pydbtest;
"""

cursor.execute(sql)
rows = cursor.fetchall()
for row in rows:
    print(row['id'], row['name'], row['price'])

db.close()

print("Selection Success")

1 Xeon Server 10000000
2 FPGA Server 34000000
3 GPU Server 20000000
4 RTX Titan 4000000
Selection Success


In [9]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

# 기존에 만든 테이블을 삭제하는 방법
# drop table 테이블이름
cursor = db.cursor(pymysql.cursors.DictCursor)

sql = """
    drop table pydbtest;
"""

cursor.execute(sql)

db.commit()
db.close()

print("Selection Success")

Selection Success


In [2]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor()

sql = """
    create table sensor_data(
        no integer not null auto_increment primary key, 
        voltage float not null, 
        current float not null, 
        vendor text null, 
        name text not null, 
        reg_date timestamp not null default now()
    )
"""

cursor.execute(sql)

db.commit()

db.close()

print("Table Creation Success")

InternalError: (1050, "Table 'sensor_data' already exists")

In [3]:
import time
import math
import pymysql
import calendar
import datetime
import numpy as np

In [4]:
sampleTime = np.arange(0, 10, 0.001)
noiseData = np.random.normal(size = len(sampleTime))

# f = 1/T
# T = 1/f
# 2 * pi * f * t = w * t
voltage = 3* np.cos(10 * sampleTime + (math.pi / 3)) + \
    2.7 * np.cos(7 * sampleTime) + \
    2.5 * np.cos(5 * noiseData * sampleTime)

current = 3 * np.cos(10 * sampleTime) + \
    2.7 * np.cos(7 * sampleTime) + \
    2.5 * np.cos(5 * noiseData * sampleTime)

In [5]:
ts = calendar.timegm(time.gmtime())
ts = (sampleTime * 1000) + ts

sampleTime = np.array(sampleTime, dtype=np.float32)
voltage = np.array(voltage, dtype=np.float32)
current = np.array(current, dtype=np.float32)

In [6]:
currTime = []

for i in range(len(sampleTime)):
    currTime.append(
        datetime.
            datetime.
                fromtimestamp(ts[i]).
                    strftime('%Y-%m-%d %H:%M:%S')
    )

In [7]:
db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor()

sql = """
    insert into sensor_data(
        voltage, current, vendor, name, reg_date
    ) values (%s, %s, %s, %s, %s)
"""

In [8]:
try:
    with db.cursor() as cursor:
        for i in range(len(sampleTime)):
            cursor.execute(sql,
                (
                    str(voltage[i]),
                    str(current[i]),
                    'FMFire',
                    'F33N21BTN7',
                    currTime[i]
                )
            )
            
        db.commit()
finally:
    db.close()
    
print("Sucess to insert 10000 Sensor Data")

Sucess to insert 10000 Sensor Data


In [2]:
from flask import Flask, jsonify

import requests
import json

In [3]:
# pip install flask_restful
from flask_restful import Resource, Api, reqparse

In [ ]:
app = Flask(__name__)
api = Api(app)

# localhost:8080/dataServer로 요청을 하면 아래함수가 동작
@app.route('/dataServer')
def PythonServerResponse():
    print("It's operate")
    summary = {'name': 'Hello', 'major': 'C++'}
    return jsonify(summary)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jul/2020 17:15:35] "GET /dataServer HTTP/1.1" 200 -


It's operate


127.0.0.1 - - [03/Jul/2020 17:44:58] "GET /dataServer HTTP/1.1" 200 -


It's operate
